## LU分解

In [1]:
import numpy as np

In [3]:
def pivoting(A):
    n = A.shape[0]
    P = np.identity(n)

    for j in range(n):
        row = max(range(j, n), key=lambda i: abs(A[i][j]))
        if j != row:
            P[[j, row]] = P[[row, j]]
    return P

In [4]:
def LUPivot_Factorization(A):
    n = A.shape[0]
    L = np.zeros([n, n])

    P = pivoting(A)
    PA = np.dot(P, A)
    
    for j in range(n):
        L[j, j] = 1
        for i in range(j+1, n):
            s = PA[i, j] / PA[j, j]
            PA[i] = s * PA[j] - PA[i]
            L[i, j] = s
    
    U = PA
    return P, L, U

### $A=\left[\begin{matrix}1 & -2 & 1\\0 & 2 & -8\\5 & 0 & -5\end{matrix}\right]$

In [2]:
A = np.array([[1,-2, 1], [0, 2, -8], [5, 0, -5]])
A

array([[ 1, -2,  1],
       [ 0,  2, -8],
       [ 5,  0, -5]])

In [5]:
p, l, u = LUPivot_Factorization(A)
print("P:{}\n".format(p))
print("L:{}\n".format(l))
print("U:{}".format(u))

P:[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]

L:[[ 1.   0.   0. ]
 [ 0.   1.   0. ]
 [ 0.2 -1.   1. ]]

U:[[ 5.  0. -5.]
 [ 0. -2.  8.]
 [-0.  0. -6.]]


### 使用 `Scipy`

In [1]:
import scipy.linalg

In [6]:
p, l, u = scipy.linalg.lu(A)
print("P:\n {}".format(p))
print("L:\n {}".format(l))
print("U:\n {}".format(u))

P:
 [[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
L:
 [[ 1.   0.   0. ]
 [ 0.   1.   0. ]
 [ 0.2 -1.   1. ]]
U:
 [[ 5.  0. -5.]
 [ 0.  2. -8.]
 [ 0.  0. -6.]]


In [7]:
# validation
np.allclose(p @ l @ u, A)

True

## 使用LU分解来解 $Ax=b$
### $A=\left[\begin{matrix}1 & -2 & 1\\0 & 2 & -8\\5 & 0 & -5\end{matrix}\right]$,  $b=\left[\begin{matrix}0\\8\\10\end{matrix}\right]$

In [8]:
def forward_substitution(L, b):
    b = np.rot90(b.T)
    n = len(L)
    y = np.zeros(n)
    y[0] = b[0]
    for k in range(1, n):
        y[k] = b[k] - L[k, 0:k] @ y[0:k]
    return y

In [9]:
def back_substitution(U, b):
    n = len(U)
    x = np.zeros(n)
    G = np.c_[U, y]
    x[n-1] = G[n-1, n] / G[n-1, n-1]
    
    for k in range(n-1, -1, -1):
        x[k] = (G[k, n] - np.dot(G[k, k+1:n], x[k+1:n])) / G[k, k]
    return x

In [13]:
b = np.array([[0], [8], [10]])
_, L, U = scipy.linalg.lu(A)
y = forward_substitution(L, b)
back_substitution(U, y)

array([ 1.,  0., -1.])

In [14]:
A = np.array([[2, -2, 3], [-2, 5, 4], [-3, 4, 5]])
b = np.array([[7], [-12], [-12]])

In [15]:
_, L, U = scipy.linalg.lu(A)
y = forward_substitution(L, b)
back_substitution(U, y)

array([ 1.74418605, -1.72093023,  0.02325581])

In [16]:
L

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.66666667,  1.        ,  0.        ],
       [-0.66666667,  0.28571429,  1.        ]])

### 使用 `Numpy`

In [11]:
x = np.linalg.solve(A, b)
x

array([[ 1.],
       [ 0.],
       [-1.]])